In [5]:
# imports
import pandas as pd
import requests
import os
from urllib3.exceptions import IncompleteRead


ModuleNotFoundError: No module named 'requests'

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
salvador_station = pd.read_csv(r"C:\Users\johnk\OneDrive\Desktop\Statistical-Modelling-Project\salvador_station.csv")

foursquare_url = "https://api.foursquare.com/v3/places/search"
API_key =os.getenv("4square_api_key")

if not API_key:
    print("API key not found. Please set the FOURSQUARE_API_KEY environment variable.")
    exit()
headers = {
    "Accept": "application/json",
    "Authorization": API_key
}


business_terms = {
    "13003": "Bars",
    "13065": "Restaurants",
    "18000": "Sports & Recreation"}

foursquare_result = []

for _, row in salvador_station.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    for term, category_name in business_terms.items():
        params ={
            "ll": f"{latitude},{longitude}",
            "radius": 1000,
            "limit": 50,
            "categories": term}
        
        try:
            response = requests.get (foursquare_url,headers = headers, params = params, timeout=300)

            if response.status_code == 200:
                data = response.json()
                venues = data.get("results", [])
                for venue in data.get("results", []):
                    foursquare_result.append({
                        "Station Name": row['Station Name'],
                        "Venue Name": venue.get('name', 'N/A'),
                        "Category": category_name,
                        "Venue Latitude": venue.get('geocodes', {}).get('main', {}).get('latitude'),
                        "Venue Longitude": venue.get('geocodes', {}).get('main', {}).get('longitude'),
                        "fsq_id": venue.get('fsq_id', 'N/A'),
                        "venue distance": venue.get('distance')
                        })
            else:
                print(f"Error for location {latitude},{longitude}: {response.status_code}")
        except IncompleteRead as e:
            print(f"Error reading response for {latitude},{longitude}: {e}")
        except Exception as e:
            print(f"An error occurred for {latitude},{longitude}: {e}") 


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

In [ ]:
foursquare_df = pd.DataFrame(foursquare_result)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
salvadore_station= pd.read_csv(r"C:\Users\johnk\OneDrive\Desktop\Statistical-Modelling-Project\salvador_station.csv")




yelp_url = "https://api.yelp.com/v3/businesses/search"
# Your API Key
api_key = os.getenv("yelp_api_key")

if not api_key:
    print("API key not found. Please set the yelp_API_KEY environment variable.")
    exit()

# Headers with Authorization
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}
yelp_result= []
business_terms = ["Restaurants","Bars","Fitness & Instruction"]


for _, row in salvadore_station.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    for term in business_terms:
# Parameters for Search
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "radius": 500,  # Radius in meters
            "term": term, 
            "limit": 50 # Number of results to return
}

# Make the Request
        response = requests.get(yelp_url, headers=headers, params=params)

# Check the Response
        if response.status_code == 200:
            data = response.json()
            for business in data.get("businesses", []):
                yelp_result.append({
                    "Station Name": row['Station Name'],
                    "Latitude": latitude,
                    "Longitude": longitude,
                    "Search Term": term,
                    "Business Name": business['name'],
                    "Business Category": term,  # Add term as category
                    "Business Rating": business['rating'],
                    "Business Address": business['location'].get('address1', 'N/A'),
                    "Business Phone": business.get('phone', 'N/A'),
                    "Business Distance (meters)": business.get('distance','N/A')})
        else:
            print(f"Error: {response.status_code}")
            print(f"Response details: {response.json()}")


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

yelp_df = pd.DataFrame(yelp_result)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [ ]:
yelp_df.sort_values(by="Business Rating", ascending = False).head(10)